In [1]:
dataname="lymphoma"

patch_size=256 #size of the tiles to extract and save in the database, must be >= to training size
stride_size=256 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
mirror_pad_size=128 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
test_set_size=.1 # what percentage of the dataset should be used as a held out validation/testing set
resize=1 #resize input images
class_names=["CLL", "FL", "MCL"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes


In [2]:
import torch
import tables

import os,sys
import glob

import PIL
import numpy as np

import cv2
import matplotlib.pyplot as plt

from sklearn import model_selection
import sklearn.feature_extraction.image
import random


In [3]:
seed = random.randrange(sys.maxsize)

In [4]:
seed = random.randrange(sys.maxsize) #get a random seed so that we can reproducibly do the cross validation setup
random.seed(seed) # set the seed
print("random seed (note down for reproducibility) :", seed)

random seed (note down for reproducibility) : 2915296278180353332


In [5]:
img_dtype = tables.UInt8Atom()  # dtype in which the images will be saved, this indicates that images will be saved as unsigned int 8 bit, i.e., [0,255]
filenameAtom = tables.StringAtom(itemsize=255) #create an atom to store the filename of the image, just incase we need it later,

In [6]:
files=glob.glob('./**/*.tif') # create a list of the files, in this case we're only interested in files which have masks so we can use supervised learning

#create training and validation stages and split the files appropriately between them
phases={}
phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))

In [11]:
filenameAtom

StringAtom(itemsize=255, shape=(), dflt=b'')

In [10]:
storage={} #holder for future pytables

block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed


for phase in phases.keys(): #now for each of the phases, we'll loop through the files
    print(phase)
    
    totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    
    hdf5_file = tables.open_file(f"./dataname_phase.pytable", mode='w') #open the respective pytable
    storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    
    storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
                                              shape=np.append([0],block_shape), 
                                              chunkshape=np.append([1],block_shape),
                                              filters=filters)
    storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
                                              shape=[0], 
                                              chunkshape=[1],
                                              filters=filters)

    
    for filei in phases[phase]: #now for each of the files
        fname=files[filei] 
        
        print(fname)
        classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
        totals[classid]+=1

        io=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
        interp_method=PIL.Image.BICUBIC


        io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
        io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

        #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
        io_arr_out=sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)

        #resize it into a ntile x patch_size x patch_size x 3
        io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)



        storage["imgs"].append(io_arr_out)
        storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        
    #lastely, we should store the number of pixels
    npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
    npixels[:]=totals
    hdf5_file.close()

train
./MCL/sj-04-6010-R3_005.tif


/home/jindo/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function extract_patches is deprecated; The function feature_extraction.image.extract_patches has been deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(msg, category=FutureWarning)


./MCL/sj-05-768_010.tif
./CLL/sj-05-3165_002.tif
./FL/sj-05-1881-R1_016.tif
./MCL/sj-05-3362-R2_015.tif
./MCL/sj-05-768_003.tif
./MCL/sj-05-3362-R2_007.tif
./CLL/sj-05-1396-R3_002.tif
./CLL/sj-03-476_007.tif
./MCL/sj-04-4525-R4_004.tif
./CLL/sj-05-3165_007.tif
./MCL/sj-04-4967-R2_003.tif
./MCL/sj-04-4525-R4_011.tif
./FL/sj-05-5389-R1_017.tif
./CLL/sj-05-3344_010.tif
./FL/sj-05-894-R3_002.tif
./FL/sj-05-5311-R1_009.tif
./CLL/sj-05-3344_004.tif
./MCL/sj-05-768_015.tif
./MCL/sj-05-768_006.tif
./CLL/sj-03-5521_003.tif
./CLL/sj-03-2810_010.tif
./MCL/sj-04-4525-R4_005.tif
./MCL/sj-05-901-R1_005.tif
./CLL/sj-03-4957_005.tif
./MCL/sj-05-3362-R2_003.tif
./MCL/sj-04-4967-R2_008.tif
./FL/sj-05-5311-R1_012.tif
./FL/sj-05-5311-R1_010.tif
./MCL/sj-05-768_011.tif
./MCL/sj-05-901-R1_010.tif
./FL/sj-05-894-R3_006.tif
./CLL/sj-03-852-R2_012.tif
./MCL/sj-05-901-R1_002.tif
./FL/sj-05-588-R1_001.tif
./CLL/sj-05-3165_011.tif
./CLL/sj-05-3165_006.tif
./MCL/sj-04-6010-R3_009.tif
./CLL/sj-05-1396-R3_001.tif
./